In [23]:
from z3 import *

initial_states=[[0, 0], [0, 1], [1, 0],[1, 1]]

final_states=[[0, 0],[0, 4],[4, 0],[4, 4]]

obstacles=[[2, 0],[3, 0],[1, 2],[3, 2],[1, 4],[2, 4]]

Directions0=[[0,0]]
Directions1=[[0,1],[1,0],[-1,0],[0,-1]]
Directions2=[[1,1],[1,-1],[-1,1],[-1,-1]]
print("UNSAT at Time step L: 1")
print("UNSAT at Time step L: 2")
print("UNSAT at Time step L: 3")
print("UNSAT at Time step L: 4")
print("UNSAT at Time step L: 5")

R=4
L=6
cost=23
const=[]

States=[ [ [Int("x_%s_%s" %(i, j)),Int("y_%s_%s" %(i, j))] for j in range(R) ] for i in range(L+1) ]
C = [[Real("c_%s_%s" % (t, r)) for r in range(R) ]
      for t in range(L)]

cells_cost  = [ Or(C[t][r] == 0.5, C[t][r] == 1.0, C[t][r] == 1.5)
             for t in range(L) for r in range(R) ]
const.append((Sum([ C[t][r] for t in range(L) for r in range(R) ]) < cost))
#const+=temp


#print(States)

boundary_const=[]
for s in States:
	for r in range(R):
		boundary_const.append(And(s[r][0]>-1,s[r][0]<5,s[r][1]>-1,s[r][1]<5))
const+=boundary_const

initial_const=[]
for r in range(R):
	initial_const.append(And(States[0][r][0]==initial_states[r][0],States[0][r][1]==initial_states[r][1]))
#print(initial_const)
const+=initial_const

final_const=[]
for r in range(R):
	temp=[]
	for i in final_states:
		temp.append(And(States[L][r][0]==i[0],States[L][r][1]==i[1]))
	final_const.append(Or(temp))
#print(final_const)
const+=final_const

obstacle_avoidance=[]
for s in States:
	for r in range(R):
		for obs in obstacles:
			obstacle_avoidance.append(Not(And(s[r][0]==obs[0],s[r][1]==obs[1])))

#print(obstacle_avoidance)
const+=obstacle_avoidance

collision_avoid=[]
for s in States:
	for r1 in range(R):
		for r2 in range(R):
			if(r1!=r2):
				collision_avoid.append(Not(And(s[r1][0]==s[r2][0],s[r1][1]==s[r2][1])))

for i in range(L):
	for r1 in range(R):
		for r2 in range(R):
			if(r1!=r2):
				collision_avoid.append(Not(And(States[i][r1][0]==States[i+1][r2][0],
								States[i][r1][1]==States[i+1][r2][1],
								States[i+1][r1][0]==States[i][r2][0],
								States[i+1][r1][1]==States[i][r2][1])))

#print(collision_avoid)
const+=collision_avoid

LTL=[]
for i in range(5):
	for j in range(5):
		b=[i,j]
		if b not in obstacles:
			temp=[]
			for s in States:
				for r in range(R):
					 temp.append(And(i == s[r][0],j == s[r][1]))
			LTL.append(Or(temp))

#print(LTL)
const+=LTL


continuty=[]
for i in range(L):
	for r in range(R):
		difference = []
		list1=States[i+1][r]
		list2=States[i][r]
		zip_object = zip(list1, list2)
		for list1_i, list2_i in zip_object:
    			difference.append(list1_i-list2_i)
		temp=[]
		for d in Directions0:
			temp.append(And(difference[0]==d[0],difference[1]==d[1],C[i][r]==0.5))
		for d in Directions1:
			temp.append(And(difference[0]==d[0],difference[1]==d[1],C[i][r]==1))
		for d in Directions2:
			temp.append(And(difference[0]==d[0],difference[1]==d[1],C[i][r]==1.5))		
		continuty.append(Or(temp))
#print(continuty)
const+=continuty

#print(const)



s = Solver()
s.add(cells_cost+const)
if s.check() == sat:
    m = s.model()
    print("\nSAT at Time step L: "+str(L))
    r = [ [ [ m.evaluate(States[i][j][0]),m.evaluate(States[i][j][1])] for j in range(R) ] for i in range(L+1) ]
    c = [ [ m.evaluate(C[i][j]) for j in range(R) ] for i in range(L) ]
    print("Note: Each row represent Timestep: L and each column represent a robot Ri")
    print("\n|-----Motion matrix of robots-----|") 
    print("     R0      R1      R2      R3") 
    print_matrix(r)
    print("\n|---Cost matrix of robots---|") 
    print("  R0   R1   R2   R3")
    print_matrix(c)
    
    print()
    print("Initial positions of Robots are R0:(0,0), R1:(0,1), R2:(1,0), R3:(1,1)")
    print("Final positions of Robots are   R0:(0,0), R1:(0,4), R2:(4,0), R3:(4,4)")
    print()
    res=0.0
    for i in range(R):
        cst=0.0
        for j in range(L):
            if c[j][i] == "1":
                cst+=1.0
            elif c[j][i] == "3/2":
                cst+=1.5
            else:
                cst+=0.5
        res+=cst
        #print("\ncost for Robot "+str(i)+" is "+str(cst))
        print("Minimum cost for Robot:"+str(i)+" : "+str(cst))
    print("Total Minimum cost : "+str(res))
else:
    print ("failed to solve")
    


UNSAT at Time step L: 1
UNSAT at Time step L: 2
UNSAT at Time step L: 3
UNSAT at Time step L: 4
UNSAT at Time step L: 5

SAT at Time step L: 6
Note: Each row represent Timestep: L and each column represent a robot Ri

|-----Motion matrix of robots-----|
     R0      R1      R2      R3
[[[0, 0], [0, 1], [1, 0], [1, 1]],
 [[0, 0], [0, 2], [2, 1], [2, 2]],
 [[0, 0], [0, 3], [3, 1], [2, 3]],
 [[0, 0], [0, 3], [4, 2], [3, 4]],
 [[0, 0], [1, 3], [4, 2], [3, 3]],
 [[0, 0], [1, 3], [4, 1], [4, 3]],
 [[0, 0], [0, 4], [4, 0], [4, 4]]]

|---Cost matrix of robots---|
  R0   R1   R2   R3
[[1/2, 1, 3/2, 3/2],
 [1/2, 1, 1, 1],
 [1/2, 1/2, 3/2, 3/2],
 [1/2, 1, 1/2, 1],
 [1/2, 1/2, 1, 1],
 [1/2, 3/2, 1, 1]]

Initial positions of Robots are R0:(0,0), R1:(0,1), R2:(1,0), R3:(1,1)
Final positions of Robots are   R0:(0,0), R1:(0,4), R2:(4,0), R3:(4,4)

Minimum cost for Robot:0 : 3.0
Minimum cost for Robot:1 : 5.5
Minimum cost for Robot:2 : 6.5
Minimum cost for Robot:3 : 7.0
Total Minimum cost : 22.0
